In [ ]:
from IPython.display import Image, display
import os

# Display the logo image from the Assets directory
image_path = os.path.join("Assets", "HeatReuseEconomicsTool_Horizontal.jpg")

# Display the image with appropriate sizing
display(Image(filename=image_path, width=600))

# Interactive Analysis Tool

## User Guide

This notebook provides an easy-to-use interface for running calculations based on temperature and flow inputs.

### Instructions:
1. Run the setup cell below by clicking the ▶️ button or pressing Shift+Enter
2. Enter your values for temperatures (T1-T4) and flows (F1-F2) in the input fields
3. Click the "Run Calculation" button to process the data
4. View the results and visualizations below the button

**Note:** If you encounter any errors, please ensure all input values are numbers.

In [1]:
# Import required libraries
import ipywidgets as widgets
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, clear_output, HTML
import io
import base64
import math
from datetime import datetime
from pathlib import Path
import os

# Set up plotting configuration
%matplotlib inline
plt.style.use('ggplot')



In [10]:
# 20250602 Lookup Functions
# Corrected Lookup Functions for Heat Reuse Tool
# These replace the existing calculation functions with proper CSV lookups

import pandas as pd
import numpy as np

def lookup_allhx_data(power, t1, temp_diff, approach):
    """
    Look up system parameters from ALLHX.csv based on user selections.
    
    Parameters:
    power (float): Power/capacity in MW (1, 2, 3, 4, 5)
    t1 (float): T1 temperature (20, 30, 45)
    temp_diff (float): Temperature difference (10, 12, 14)
    approach (float): Approach value (2, 3, 5)
    
    Returns:
    dict: System parameters including F1, F2, T3, T4, costs, etc.
    """
    global csv_data
    
    # Calculate T2 from T1 + temperature difference
    t2 = t1 + temp_diff
    
    # Get the ALLHX dataframe
    if 'ALLHX' not in csv_data:
        print("Error: ALLHX.csv not loaded")
        return None
    
    df = csv_data['ALLHX']
    
    # Filter the dataframe based on user inputs
    # Note: Assuming 'wha' column contains power, adjust column names as needed
    filtered_df = df[
        (df['wha'] == power) &
        (df['T1'] == t1) &
        (df['itdt'] == temp_diff) &  # itdt = temperature difference
        (df['T2'] == t2) &
        (df['TCSapp'] == approach)  # TCS approach
    ]
    
    if filtered_df.empty:
        print(f"No data found for: Power={power}MW, T1={t1}°C, TempDiff={temp_diff}°C, Approach={approach}")
        return None
    
    # Get the first matching row
    row = filtered_df.iloc[0]
    
    # Extract system parameters
    result = {
        'power': power,
        'T1': t1,
        'T2': t2,
        'T3': row['T3'],
        'T4': row['T4'],
        'F1': row['F1'],  # TCS Flow Rate
        'F2': row['F2'],  # FWS Flow Rate
        'approach_tcs': row['TCSapp'],
        'approach_fws': row['FWSapp'],
        'hx_unit': row['Unit'],
        'hx_cost': row['costHX'],
        'hx_area': row['areaHX'],
        'hx_weight': row['Hxweight'],
        'co2_footprint': row['CO2_Footprint']
    }
    
    return result

def get_system_sizing(system_data):
    """
    Calculate system sizing based on flows and power.
    
    Parameters:
    system_data (dict): Result from lookup_allhx_data()
    
    Returns:
    dict: Sizing information including pipe sizes, room size, etc.
    """
    if not system_data:
        return None
    
    # Get pipe sizes based on flow rates
    pipe_size_f1 = get_lookup_value("PIPSZ", system_data['F1'], 0, 1)
    pipe_size_f2 = get_lookup_value("PIPSZ", system_data['F2'], 0, 1)
    
    # Get room size based on power (using MW conversion)
    mw_divided = system_data['power']  # Already in MW
    room_size = get_lookup_value("ROOM", mw_divided)
    
    sizing_data = {
        'pipe_size_f1': pipe_size_f1,
        'pipe_size_f2': pipe_size_f2,
        'room_size': room_size,
        'primary_pipe_size': max(pipe_size_f1 or 0, pipe_size_f2 or 0)  # Use larger pipe size
    }
    
    return sizing_data

def calculate_system_costs(system_data, sizing_data):
    """
    Calculate all system costs based on system and sizing data.
    
    Parameters:
    system_data (dict): Result from lookup_allhx_data()
    sizing_data (dict): Result from get_system_sizing()
    
    Returns:
    dict: Complete cost breakdown
    """
    if not system_data or not sizing_data:
        return None
    
    # Get primary pipe size for cost calculations
    primary_pipe_size = sizing_data['primary_pipe_size']
    room_size = sizing_data['room_size']
    
    # Calculate pipe costs
    pipe_cost_per_meter = get_lookup_value("PIPCOST", primary_pipe_size, 0, 1)  # Sch 40
    total_pipe_length = room_size * 3  # Based on meeting notes: 3x room dimension
    total_pipe_cost = pipe_cost_per_meter * total_pipe_length if pipe_cost_per_meter else 0
    
    # Calculate valve costs (assuming 8 valves as mentioned in meeting)
    control_valve_cost = get_exact_match("CVALV", str(int(primary_pipe_size)), 0, 1) or 0
    isolation_valve_cost = get_exact_match("IVALV", str(int(primary_pipe_size)), 0, 1) or 0
    total_valve_cost = (control_valve_cost + isolation_valve_cost) * 4  # 4 of each type
    
    # Heat exchanger cost (from ALLHX data)
    hx_cost = system_data['hx_cost']
    
    # Pump cost (placeholder as mentioned in meeting notes)
    pump_cost = system_data['power'] * 5000  # Estimated €5000 per MW
    
    # Installation cost (constant as mentioned in specs)
    installation_cost = 10000  # Placeholder
    
    # Total system cost
    total_cost = total_pipe_cost + total_valve_cost + hx_cost + pump_cost + installation_cost
    
    cost_data = {
        'pipe_cost_per_meter': pipe_cost_per_meter,
        'total_pipe_length': total_pipe_length,
        'total_pipe_cost': total_pipe_cost,
        'control_valve_cost': control_valve_cost,
        'isolation_valve_cost': isolation_valve_cost,
        'total_valve_cost': total_valve_cost,
        'hx_cost': hx_cost,
        'pump_cost': pump_cost,
        'installation_cost': installation_cost,
        'total_cost': total_cost
    }
    
    return cost_data

def get_complete_system_analysis(power, t1, temp_diff, approach):
    """
    Main function to get complete system analysis from user inputs.
    
    Parameters:
    power (float): Power/capacity in MW (1, 2, 3, 4, 5)
    t1 (float): T1 temperature (20, 30, 45)
    temp_diff (float): Temperature difference (10, 12, 14)
    approach (float): Approach value (2, 3, 5)
    
    Returns:
    dict: Complete system analysis including parameters, sizing, and costs
    """
    # Step 1: Get basic system data from ALLHX
    system_data = lookup_allhx_data(power, t1, temp_diff, approach)
    if not system_data:
        return None
    
    # Step 2: Calculate sizing
    sizing_data = get_system_sizing(system_data)
    if not sizing_data:
        return None
    
    # Step 3: Calculate costs
    cost_data = calculate_system_costs(system_data, sizing_data)
    if not cost_data:
        return None
    
    # Combine all data
    complete_analysis = {
        'system': system_data,
        'sizing': sizing_data,
        'costs': cost_data,
        'summary': {
            'power_mw': system_data['power'],
            't1_celsius': system_data['T1'],
            't2_celsius': system_data['T2'],
            't3_celsius': system_data['T3'],
            't4_celsius': system_data['T4'],
            'f1_flow': system_data['F1'],
            'f2_flow': system_data['F2'],
            'pipe_size': sizing_data['primary_pipe_size'],
            'room_size': sizing_data['room_size'],
            'total_cost_eur': round(cost_data['total_cost'])
        }
    }
    
    return complete_analysis

# Validation function for user inputs
def validate_user_inputs(power, t1, temp_diff, approach):
    """
    Validate that user inputs are within acceptable ranges.
    
    Returns:
    list: List of error messages, empty if all inputs are valid
    """
    errors = []
    
    # Validate power
    if power not in [1, 2, 3, 4, 5]:
        errors.append(f"Power must be 1, 2, 3, 4, or 5 MW (got {power})")
    
    # Validate T1
    if t1 not in [20, 30, 45]:
        errors.append(f"T1 must be 20, 30, or 45°C (got {t1})")
    
    # Validate temperature difference
    if temp_diff not in [10, 12, 14]:
        errors.append(f"Temperature difference must be 10, 12, or 14°C (got {temp_diff})")
    
    # Validate approach
    if approach not in [2, 3, 5]:
        errors.append(f"Approach must be 2, 3, or 5 (got {approach})")
    
    return errors

In [2]:
# Dictionary to store all dataframes
csv_data = {}

# Function to get a specific dataframe
def get_dataframe(csv_name):
    """
    Get a specific dataframe by name.
    
    Parameters:
    csv_name (str): Name of the CSV file (case-insensitive)
    
    Returns:
    pandas.DataFrame: The dataframe, or None if not found
    """
    global csv_data
    
    # Normalize the CSV name
    csv_name = csv_name.upper()
    
    # Check if the CSV has been loaded
    if csv_name not in csv_data:
        print(f"CSV '{csv_name}' not found. Available CSVs: {list(csv_data.keys())}")
        return None
    
    return csv_data[csv_name]

def load_csv_files(data_dir="Data"):
    """
    Load all CSV files from the specified directory.
    
    Parameters:
    data_dir (str): Path to the directory containing CSV files
    
    Returns:
    dict: Dictionary of dataframes with normalized names as keys
    """
    global csv_data
    
    try:
        # Get all CSV files in the directory
        csv_files = [f for f in os.listdir(data_dir) if f.endswith('.csv')]
        
        # Load each CSV file into a dataframe
        for file in csv_files:
            # Create a normalized name for the dataframe (without .csv extension, uppercase)
            df_name = os.path.splitext(file)[0].upper()
            file_path = os.path.join(data_dir, file)
            
            try:
                # Try to read the CSV file
                csv_data[df_name] = pd.read_csv(file_path)
                print(f"Loaded: {file} as {df_name}")
            except Exception as e:
                # Try with different separators if automatic detection fails
                try:
                    csv_data[df_name] = pd.read_csv(file_path, sep=';')
                    print(f"Loaded: {file} as {df_name} (using semicolon separator)")
                except:
                    try:
                        csv_data[df_name] = pd.read_csv(file_path, sep='\t')
                        print(f"Loaded: {file} as {df_name} (using tab separator)")
                    except Exception as e2:
                        print(f"Failed to load {file}: {e2}")
        
        return csv_data
    
    except Exception as e:
        print(f"Error loading CSV files: {e}")
        return {}

def get_lookup_value(csv_name, lookup_value, col_index_lookup=0, col_index_return=1):
    """
    Look up a value in a CSV file based on finding the first value 
    in col_index_lookup that is >= lookup_value, then return the 
    corresponding value from col_index_return.
    
    Parameters:
    csv_name (str): Name of the CSV file (case-insensitive)
    lookup_value: Value to look up (will be compared against col_index_lookup)
    col_index_lookup (int): Index of column to search in (default: 0)
    col_index_return (int/str/list): Index of column(s) to return value from (default: 1)
                                    Can be an integer, column name, or list of integers/names
    
    Returns:
    The value from col_index_return corresponding to the first row where
    col_index_lookup >= lookup_value, or None if not found.
    If col_index_return is a list, returns a dictionary with column names/indices as keys.
    """
    global csv_data
    
    # Normalize the CSV name
    csv_name = csv_name.upper()
    
    # Check if the CSV has been loaded
    if csv_name not in csv_data:
        print(f"CSV '{csv_name}' not found. Available CSVs: {list(csv_data.keys())}")
        return None
    
    # Get the dataframe
    df = csv_data[csv_name]
    
    # Get column name for lookup column
    lookup_col = df.columns[col_index_lookup] if isinstance(col_index_lookup, int) else col_index_lookup
    
    # Sort the dataframe by the lookup column to ensure proper comparison
    df_sorted = df.sort_values(by=lookup_col)
    
    # Find rows where lookup column >= lookup_value
    matching_rows = df_sorted[df_sorted[lookup_col] >= lookup_value]
    
    # If no matching rows, return None
    if matching_rows.empty:
        return None
    
    # Get the first matching row (which will be the smallest value >= lookup_value)
    matched_row = matching_rows.iloc[0]
    
    # Handle different return column specifications
    if isinstance(col_index_return, (list, tuple)):
        # Return multiple columns as a dictionary
        result = {}
        for col in col_index_return:
            col_name = df.columns[col] if isinstance(col, int) else col
            if col_name in df.columns:
                result[col_name] = matched_row[col_name]
            else:
                print(f"Warning: Column '{col_name}' not found in '{csv_name}'")
        return result
    else:
        # Return a single column
        return_col = df.columns[col_index_return] if isinstance(col_index_return, int) else col_index_return
        if return_col in df.columns:
            return matched_row[return_col]
        else:
            print(f"Column '{return_col}' not found in '{csv_name}'. Available columns: {list(df.columns)}")
            return None

def get_lookup_value_by_name(csv_name, lookup_value, lookup_col_name, return_col_name):
    """
    Look up a value in a CSV file based on finding the first value 
    in lookup_col_name that is >= lookup_value, then return the 
    corresponding value from return_col_name.
    
    Parameters:
    csv_name (str): Name of the CSV file (case-insensitive)
    lookup_value: Value to look up (will be compared against lookup_col_name)
    lookup_col_name (str): Name of column to search in
    return_col_name (str/list): Name of column(s) to return value from.
                               Can be a single column name or list of names.
    
    Returns:
    The value from return_col_name corresponding to the first row where
    lookup_col_name >= lookup_value, or None if not found.
    If return_col_name is a list, returns a dictionary with column names as keys.
    """
    global csv_data
    
    # Normalize the CSV name
    csv_name = csv_name.upper()
    
    # Check if the CSV has been loaded
    if csv_name not in csv_data:
        print(f"CSV '{csv_name}' not found. Available CSVs: {list(csv_data.keys())}")
        return None
    
    # Get the dataframe
    df = csv_data[csv_name]
    
    # Check if lookup column exists
    if lookup_col_name not in df.columns:
        print(f"Column '{lookup_col_name}' not found in '{csv_name}'. Available columns: {list(df.columns)}")
        return None
    
    # If return_col_name is a list, check all columns exist
    if isinstance(return_col_name, (list, tuple)):
        for col in return_col_name:
            if col not in df.columns:
                print(f"Column '{col}' not found in '{csv_name}'. Available columns: {list(df.columns)}")
                # Continue anyway, will just skip this column
    else:
        # Single column - check it exists
        if return_col_name not in df.columns:
            print(f"Column '{return_col_name}' not found in '{csv_name}'. Available columns: {list(df.columns)}")
            return None
    
    # Sort the dataframe by the lookup column to ensure proper comparison
    df_sorted = df.sort_values(by=lookup_col_name)
    
    # Find rows where lookup column >= lookup_value
    matching_rows = df_sorted[df_sorted[lookup_col_name] >= lookup_value]
    
    # If no matching rows, return None
    if matching_rows.empty:
        return None
    
    # Get the first matching row (which will be the smallest value >= lookup_value)
    matched_row = matching_rows.iloc[0]
    
    # Handle different return column specifications
    if isinstance(return_col_name, (list, tuple)):
        # Return multiple columns as a dictionary
        result = {}
        for col in return_col_name:
            if col in df.columns:
                result[col] = matched_row[col]
        return result
    else:
        # Return a single column
        return matched_row[return_col_name]

def get_exact_match(csv_name, lookup_value, lookup_col_index=0, return_col_index=1):
    """
    Look up a value in a CSV file based on finding an exact match
    in lookup_col_index, then return the corresponding value from return_col_index.
    
    Parameters:
    csv_name (str): Name of the CSV file (case-insensitive)
    lookup_value: Value to look up (must match exactly)
    lookup_col_index (int/str): Index or name of column to search in (default: 0)
    return_col_index (int/str/list): Index or name of column(s) to return value from (default: 1)
                                    Can be an integer, column name, or list of integers/names
    
    Returns:
    The value from return_col_index corresponding to the row where
    lookup_col_index == lookup_value, or None if not found.
    If return_col_index is a list, returns a dictionary with column names/indices as keys.
    """
    global csv_data
    
    # Normalize the CSV name
    csv_name = csv_name.upper()
    
    # Check if the CSV has been loaded
    if csv_name not in csv_data:
        print(f"CSV '{csv_name}' not found. Available CSVs: {list(csv_data.keys())}")
        return None
    
    # Get the dataframe
    df = csv_data[csv_name]
    
    # Get column name for lookup column
    lookup_col = df.columns[lookup_col_index] if isinstance(lookup_col_index, int) else lookup_col_index
    
    # Find rows where lookup column == lookup_value
    matching_rows = df[df[lookup_col] == lookup_value]
    
    # If no matching rows, return None
    if matching_rows.empty:
        return None
    
    # Get the first matching row
    matched_row = matching_rows.iloc[0]
    
    # Handle different return column specifications
    if isinstance(return_col_index, (list, tuple)):
        # Return multiple columns as a dictionary
        result = {}
        for col in return_col_index:
            col_name = df.columns[col] if isinstance(col, int) else col
            if col_name in df.columns:
                result[col_name] = matched_row[col_name]
            else:
                print(f"Warning: Column '{col_name}' not found in '{csv_name}'")
        return result
    else:
        # Return a single column
        return_col = df.columns[return_col_index] if isinstance(return_col_index, int) else return_col_index
        if return_col in df.columns:
            return matched_row[return_col]
        else:
            print(f"Column '{return_col}' not found in '{csv_name}'. Available columns: {list(df.columns)}")
            return None

def show_csv_info(csv_name=None):
    """
    Display information about loaded CSV files.
    
    Parameters:
    csv_name (str, optional): Name of specific CSV to display info for.
                             If None, displays info for all CSVs.
    """
    global csv_data
    
    if csv_name:
        # Normalize the CSV name
        csv_name = csv_name.upper()
        
        # Check if the CSV has been loaded
        if csv_name not in csv_data:
            print(f"CSV '{csv_name}' not found. Available CSVs: {list(csv_data.keys())}")
            return
        
        # Display info for the specified CSV
        df = csv_data[csv_name]
        print(f"\n=== CSV: {csv_name} ===")
        print(f"Shape: {df.shape}")
        print("\nColumns:")
        for i, col in enumerate(df.columns):
            print(f"  {i}: {col}")
        print("\nFirst 5 rows:")
        print(df.head())
    else:
        # Display info for all CSVs
        for name, df in csv_data.items():
            print(f"\n=== CSV: {name} ===")
            print(f"Shape: {df.shape}")
            print("\nColumns:")
            for i, col in enumerate(df.columns):
                print(f"  {i}: {col}")
            print("\nFirst 5 rows:")
            print(df.head())

# Example usage
# First load all CSVs
# data_dir = "Data"  # Update this to your data directory
# load_csv_files(data_dir)

# Example of how to use the lookup functions
# pipe_size = 2.0
# pipe_cost = get_lookup_value("PIPCOST", pipe_size)
# print(f"Cost for pipe size {pipe_size}: {pipe_cost}")

# Example of how to use exact match lookup
# valve_type = "GATE"
# valve_cost = get_exact_match("CVALV", valve_type)
# print(f"Cost for valve type {valve_type}: {valve_cost}")

# Example of calculations using values from CSVs
# def calculate_system_cost(pipe_size, pipe_length, valve_type=None):
#     # Get pipe cost per unit length
#     pipe_cost_per_unit = get_lookup_value("PIPCOST", pipe_size)
#     total_pipe_cost = pipe_cost_per_unit * pipe_length
#     
#     # Add valve cost if specified
#     total_valve_cost = 0
#     if valve_type:
#         valve_cost = get_exact_match("CVALV", valve_type)
#         if valve_cost:
#             total_valve_cost = valve_cost
#     
#     # Calculate total cost
#     total_cost = total_pipe_cost + total_valve_cost
#     return total_cost
# 
# # Calculate system cost
# system_cost = calculate_system_cost(2.0, 100, "GATE")
# print(f"Total system cost: ${system_cost:.2f}")

load_csv_files()
# show_csv_info()

Loaded: ALLHX.csv as ALLHX
Loaded: CVALV.csv as CVALV
Loaded: HX.csv as HX
Loaded: IVALV.csv as IVALV
Loaded: JOINTS.csv as JOINTS
Loaded: MW Price Data.csv as MW PRICE DATA
Loaded: PIPCOST.csv as PIPCOST
Loaded: PIPSZ.csv as PIPSZ
Loaded: ROOM.csv as ROOM


{'ALLHX':      wha  T1 itdt  T2 TCSapp     F1  T4  T3     F2 FWSapp  ... Unnamed: 15  \
 0    1.0  20   10  30      2  1,493  18  28  1,440      2  ...         NaN   
 1    1.0  20   10  30      3  1,493  17  27  1,440      3  ...         NaN   
 2    1.0  20   10  30      5  1,493  15  25  1,440      5  ...         NaN   
 3    1.0  20   12  32      2  1,244  18  30  1,200      2  ...         NaN   
 4    1.0  20   12  32      3  1,244  17  29  1,200      3  ...         NaN   
 ..   ...  ..  ...  ..    ...    ...  ..  ..    ...    ...  ...         ...   
 131  5.0  45   12  57      5  6,209  40  52  6,061      5  ...         NaN   
 132  5.0  45   14  59      2  5,322  43  57  5,195      2  ...         NaN   
 133  5.0  45   14  59      3  5,322  42  56  5,195      3  ...         NaN   
 134  5.0  45   14  59      5  5,322  40  54  5,195      5  ...         NaN   
 135    A   B   DT   C     D       Z   Z   Z      Z      Z  ...         NaN   
 
     Unnamed: 16 Unnamed: 17 Unnamed: 18 

In [3]:
# Formulas
def mw_divd(F1, T1, T2):
    mw_value = get_MW(F1, T1, T2)
    result = mw_value / 1000000
    rounded = round(result, 2)
    return rounded


def get_itdt(T1,T2):
    # Temperature Difference Across IT side of Heat Exchanger
    x=T2-T1
    # print(x)
    return T2-T1

def get_PipeSize_Suggested(F1):
    x=get_lookup_value("PIPSZ",F1)
    return x

def get_MW(F1, T1,T2):
    # F1 = l/m
    # l/m / 60 seconds * 4186 J/kg * itdt
    itdt = get_itdt(T1,T2)
    formula = F1 / 60 * 4186 * itdt
    return formula

def get_PipeCost_perMeter(F1):
    pss = get_PipeSize_Suggested(F1)
    pcpm = get_lookup_value("PIPCOST",pss,1)
    return pcpm
    
def get_PipeLength(F1,T1,T2):
    mw = mw_divd(F1, T1, T2)
    pipelength = get_lookup_value("ROOM",mw)    
    return pipelength

def get_PipeCost_Total(F1,T1,T2):
    ttl=get_PipeCost_perMeter(F1)*get_PipeLength(F1,T1,T2)
    return ttl
    

In [4]:
# Validation Functions
# Updated Validation Functions
def validate_inputs(t1, t2, t3, t4, f1, f2):
    """Validate that all inputs are within acceptable ranges"""
    errors = []
    
    # Define reasonable limits directly in the function
    min_temp = 0.0      # Minimum temperature (°C)
    max_temp = 100.0    # Maximum temperature (°C)
    min_flow_F = 100    # Minimum flow rate (l/m)
    max_flow_F = 20000  # Maximum flow rate (l/m)
    
    # Validate temperature inputs
    for name, value in [('T1', t1), ('T2', t2), ('T3', t3), ('T4', t4)]:
        if value < min_temp:
            errors.append(f"{name} is too low (minimum: {min_temp}°C)")
        elif value > max_temp:
            errors.append(f"{name} is too high (maximum: {max_temp}°C)")
    
    # Validate flow inputs
    for name, value in [('F1', f1), ('F2', f2)]:
        if value < min_flow_F:
            errors.append(f"{name} is too low (minimum: {min_flow_F} l/m)")
        elif value > max_flow_F:
            errors.append(f"{name} is too high (maximum: {max_flow_F} l/m)")
    
    return errors

In [5]:
# Functions for graphical elements
def fig_to_base64(fig):
    """Convert a matplotlib figure to a base64 encoded image"""
    buf = io.BytesIO()
    fig.savefig(buf, format='png', bbox_inches='tight')
    buf.seek(0)
    return base64.b64encode(buf.getvalue()).decode('utf-8')


def format_results_html(t1, t2, t3, t4, f1, f2):
    """Format calculation results as an HTML report"""
    # Perform the actual calculations that work
    itdt = get_itdt(t1, t2)
    pss = get_PipeSize_Suggested(f1)
    mw = get_MW(f1, t1, t2)
    mw_divided = mw_divd(f1, t1, t2)
    pcpm = get_PipeCost_perMeter(f1)
    pl = get_PipeLength(f1, t1, t2)
    ttl = int(get_PipeCost_Total(f1, t1, t2))
    
    # Print calculation details to console
    print("itdt: ", itdt)
    print("Pipe Size Suggested: ", pss)
    print("MW: ", mw)
    print("MW Divided: ", mw_divided)
    print("Pipe Cost per Meter: ", pcpm)
    print("Pipe Length: ", pl)
    print("")
    print("Total Cost: €", ttl)
    
    # Create a simple bar chart showing the key results
    fig, axs = plt.subplots(2, 2, figsize=(12, 8))
    
    # 1. Temperature comparison chart
    temps = ['T1', 'T2', 'T3', 'T4']
    temp_values = [t1, t2, t3, t4]
    axs[0, 0].bar(temps, temp_values, color=['#ff9999', '#ff9999', '#66b3ff', '#66b3ff'])
    axs[0, 0].set_title('Temperature Readings')
    axs[0, 0].set_ylabel('Temperature (°C)')
    for i, v in enumerate(temp_values):
        axs[0, 0].text(i, v + 1, f"{v:.1f}", ha='center')
    
    # 2. Flow comparison chart
    flows = ['F1 (TCS)', 'F2 (FWS)']
    flow_values = [f1, f2]
    axs[0, 1].bar(flows, flow_values, color=['#99ff99', '#ffcc99'])
    axs[0, 1].set_title('Flow Measurements')
    axs[0, 1].set_ylabel('Flow Rate (l/m)')
    for i, v in enumerate(flow_values):
        axs[0, 1].text(i, v + max(flow_values)*0.02, f"{v:.0f}", ha='center')
    
    # 3. Cost breakdown
    cost_items = ['Pipe Cost\nper Meter', 'Pipe Length', 'Total Cost']
    cost_values = [pcpm, pl, ttl]
    colors = ['#ffcc99', '#99ccff', '#ff9999']
    axs[1, 0].bar(cost_items, cost_values, color=colors)
    axs[1, 0].set_title('Cost Analysis')
    axs[1, 0].set_ylabel('Cost/Length (€ or m)')
    for i, v in enumerate(cost_values):
        axs[1, 0].text(i, v + max(cost_values)*0.02, f"{v:.1f}", ha='center')
    
    # 4. System metrics
    metrics = ['Temperature\nDifference (itdt)', 'Megawatts', 'Pipe Size']
    metric_values = [itdt, mw_divided, pss]
    axs[1, 1].bar(metrics, metric_values, color=['#cc99ff', '#99ffcc', '#ffcc99'])
    axs[1, 1].set_title('System Metrics')
    axs[1, 1].set_ylabel('Values (°C, MW, Size)')
    for i, v in enumerate(metric_values):
        axs[1, 1].text(i, v + max(metric_values)*0.02, f"{v:.2f}", ha='center')
    
    plt.tight_layout()
    
    # Convert chart to base64
    chart_img = fig_to_base64(fig)
    plt.close(fig)  # Close the figure to free memory
    
    # Generate timestamp
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    # Create HTML report with actual calculated values
    html = f"""
    <div style="font-family: Arial, sans-serif; max-width: 900px; margin: 0 auto; padding: 20px; background-color: #f9f9f9; border-radius: 10px; box-shadow: 0 0 10px rgba(0,0,0,0.1);">
        <h2 style="color: #333; border-bottom: 2px solid #ddd; padding-bottom: 10px;">Heat Exchanger Calculation Results</h2>
        <p style="color: #777; font-style: italic;">Generated: {timestamp}</p>
        
        <div style="display: flex; justify-content: space-between; margin: 20px 0;">
            <div style="flex: 1; background-color: white; padding: 15px; border-radius: 5px; margin-right: 10px; box-shadow: 0 0 5px rgba(0,0,0,0.05);">
                <h3 style="color: #444; margin-top: 0;">Input Parameters</h3>
                <table style="width: 100%; border-collapse: collapse;">
                    <tr><td style="padding: 5px; font-weight: bold;">T1 (Outlet to TCS):</td><td style="padding: 5px;">{t1} °C</td></tr>
                    <tr><td style="padding: 5px; font-weight: bold;">T2 (Inlet from TCS):</td><td style="padding: 5px;">{t2} °C</td></tr>
                    <tr><td style="padding: 5px; font-weight: bold;">T3 (Outlet to Consumer):</td><td style="padding: 5px;">{t3} °C</td></tr>
                    <tr><td style="padding: 5px; font-weight: bold;">T4 (Inlet from Consumer):</td><td style="padding: 5px;">{t4} °C</td></tr>
                    <tr><td style="padding: 5px; font-weight: bold;">F1 (TCS Flow Rate):</td><td style="padding: 5px;">{f1} l/m</td></tr>
                    <tr><td style="padding: 5px; font-weight: bold;">F2 (FWS Flow Rate):</td><td style="padding: 5px;">{f2} l/m</td></tr>
                </table>
            </div>
            <div style="flex: 1; background-color: white; padding: 15px; border-radius: 5px; box-shadow: 0 0 5px rgba(0,0,0,0.05);">
                <h3 style="color: #444; margin-top: 0;">Calculated Results</h3>
                <table style="width: 100%; border-collapse: collapse;">
                    <tr><td style="padding: 5px; font-weight: bold;">Temperature Difference (itdt):</td><td style="padding: 5px;">{itdt} °C</td></tr>
                    <tr><td style="padding: 5px; font-weight: bold;">Power (MW):</td><td style="padding: 5px; color: #4CAF50; font-weight: bold;">{mw_divided} MW</td></tr>
                    <tr><td style="padding: 5px; font-weight: bold;">Suggested Pipe Size:</td><td style="padding: 5px;">{pss}</td></tr>
                    <tr><td style="padding: 5px; font-weight: bold;">Pipe Cost per Meter:</td><td style="padding: 5px;">€{pcpm:.2f}/m</td></tr>
                    <tr><td style="padding: 5px; font-weight: bold;">Required Pipe Length:</td><td style="padding: 5px;">{pl} m</td></tr>
                    <tr><td style="padding: 5px; font-weight: bold; color: #f44336;">Total Pipe Cost:</td><td style="padding: 5px; color: #f44336; font-weight: bold; font-size: 18px;">€{ttl:,}</td></tr>
                </table>
            </div>
        </div>
        
        <div style="background-color: white; padding: 15px; border-radius: 5px; margin-top: 20px; box-shadow: 0 0 5px rgba(0,0,0,0.05);">
            <h3 style="color: #444; margin-top: 0;">System Analysis Charts</h3>
            <img src="data:image/png;base64,{chart_img}" style="width: 100%; max-width: 800px; display: block; margin: 0 auto;">
        </div>
        
        <div style="margin-top: 20px; font-size: 12px; color: #777; text-align: center;">
            Analysis performed using the Interactive Heat Exchanger Analysis Tool
        </div>
    </div>
    """
    
    return html

In [6]:
# Create the user interface

# Title and style for the input form
display(HTML("""
<div style="background-color: #f0f0f0; padding: 10px; border-radius: 5px; margin-bottom: 10px;">
    <h3 style="margin-top: 0;">Parameter Input Form</h3>
    <p style="margin-bottom: 0;">Enter values for temperatures and flows, then click "Run Calculation"</p>
</div>
"""))

# Create input widgets with helpful descriptions
style = {'description_width': '120px'}
layout = widgets.Layout(width='400px')

t1_widget = widgets.FloatText(
    value=31.0,
    description='T1 (Outlet to TCS Temperature):',
    tooltip='Temperature at Outlet to TCS (°C)',
    style=style, layout=layout
)

t2_widget = widgets.FloatText(
    value=41.9,
    description='T2 (Inlet from TCS Temperature):',
    tooltip='Temperature at Inlet from TCS (°C)',
    style=style, layout=layout
)

t3_widget = widgets.FloatText(
    value=35,
    description='T3 (Outlet to Consumer Temperature):',
    tooltip='Temperature at Outlet to Consumer (°C)',
    style=style, layout=layout
)

t4_widget = widgets.FloatText(
    value=25,
    description='T4 (Inlet from Consumer Temperature):',
    tooltip='Temperature at Inlet from Consumer (°C)',
    style=style, layout=layout
)

f1_widget = widgets.FloatText(
    value=2100,
    description='F1 (TCS Flow Rate):',
    tooltip='Flow rate to Data Hall (l/m)',
    style=style, layout=layout
)

f2_widget = widgets.FloatText(
    value=2000,
    description='F2 (FWS Flow Rate):',
    tooltip='Flow rate to Consumer (l/m)',
    style=style, layout=layout
)

# Create output area for results and error messages
output_area = widgets.Output()

# Run calculation button with styling
button = widgets.Button(
    description='Run Calculation',
    button_style='success',
    tooltip='Click to process the inputs and generate results',
    icon='calculator',
    layout=widgets.Layout(width='200px', height='40px')
)

# Function to handle button click
def on_button_click(b):
    with output_area:
        clear_output()
        
        # Get values from widgets
        try:
            t1 = float(t1_widget.value)
            t2 = float(t2_widget.value)
            t3 = float(t3_widget.value)
            t4 = float(t4_widget.value)
            f1 = float(f1_widget.value)
            f2 = float(f2_widget.value)
        except ValueError:
            display(HTML("""
            <div style="background-color: #ffe6e6; color: #990000; padding: 10px; border-radius: 5px; margin: 10px 0;">
                <strong>Error:</strong> All values must be numbers. Please check your inputs.
            </div>
            """))
            return
        
        # Validate inputs
        errors = validate_inputs(t1, t2, t3, t4, f1, f2)
        if errors:
            error_list = "<br>".join([f"• {error}" for error in errors])
            display(HTML(f"""
            <div style="background-color: #ffe6e6; color: #990000; padding: 10px; border-radius: 5px; margin: 10px 0;">
                <strong>Input Validation Errors:</strong><br>
                {error_list}
            </div>
            """))
            return
        
        # Generate and display results directly, without the intermediate display
        try:
            html_results = format_results_html(t1, t2, t3, t4, f1, f2)
            display(HTML(html_results))
            # output_area.append_display_data(HTML(html_results))
        except Exception as e:
            display(HTML(f"""
            <div style="background-color: #ffe6e6; color: #990000; padding: 10px; border-radius: 5px; margin: 10px 0;">
                <strong>Calculation Error:</strong><br>
                An error occurred during calculation: {str(e)}
            </div>
            """))
            
# Connect the button click handler
button.on_click(on_button_click)

# Display the widgets in a organized layout
input_box = widgets.VBox([
    widgets.HBox([widgets.VBox([t1_widget, t2_widget, t3_widget]), widgets.VBox([t4_widget, f1_widget, f2_widget])]),
    widgets.HBox([button]),
    output_area
], layout=widgets.Layout(border='1px solid #ddd', padding='10px', margin='10px 0'))

display(input_box)